In [1]:
import pandas as pd
data=pd.read_csv('data_2018_11_27.csv')

In [2]:
label=pd.read_csv('label.csv',header=None)

In [3]:
label.columns=['hash','ismal']

In [4]:
merge=pd.merge(data,label,on='hash')

In [5]:
X=merge.copy()
del X['hash']
del X['ismal']

In [6]:
Y=merge['ismal']

# xgb boost

## - 이전 베스트 파라미터

In [ ]:
'''
 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, colsample_bytreeb=0.8, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
       nthread=1, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

 Best hyperparameters:
{'colsample_bytreeb': 0.8, 'gamma': 0, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 900, 'subsample': 0.8}
'''

In [68]:

#XGB부스트 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


params = {
        'n_estimators' : [600,700,800,900,1000], #중요
        'min_child_weight': [1] #1,5 # 1이 대부분 좋다..
        'gamma': [0], #  1, 1.5, 2, 5 - 큰 차이 없음
        'subsample': [0.8], #06,0.8,1.0 -> 큰 차이 없음
        'colsample_bytreeb' : [ 0.8], #06,0.8,1.0 -> 큰 차이 없음
        'max_depth': [5,7,10] #중요
        }

# 여기서 돌리고 최적 나오면 그 parameter 근처 값들로 다시 그리드 진행하면 될듯 
# https://www.programcreek.com/python/example/99824/xgboost.XGBClassifier 참고하기 좋은 자료

xgb = XGBClassifier(learning_rate=0.1, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)


grid_search1 = GridSearchCV(xgb, param_grid=params, scoring='accuracy', n_jobs=-1, cv=2, verbose=3 )
grid_search1.fit(X, Y)

print('\n Best estimator:')

print(grid_search1.best_estimator_)
print('\n Best hyperparameters:')
print(grid_search1.best_params_)






Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 21.5min finished



 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, colsample_bytreeb=0.8, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
       nthread=1, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

 Best hyperparameters:
{'colsample_bytreeb': 0.8, 'gamma': 0, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 900, 'subsample': 0.8}


In [69]:
grid_search1.best_score_

0.9523

# randomforest

In [18]:
#best parameter
#{'max_depth': 90, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

In [61]:




# RANDOMFOREST
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]

# Number of features to consider at every split
max_features = ['sqrt']

# Maximum number of levels in tree
max_depth = [60,70,80,90,100] #40,50
#int(x) for x in np.linspace(10, 30, num = 3)
# Minimum number of samples required to split a node
min_samples_split = [2] #10 

# Minimum number of samples required at each leaf node
min_samples_leaf = [1] #4


# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

# Create a based model
clf = RandomForestClassifier()

# Instantiate the grid search model
grid_search2 = GridSearchCV(estimator = clf, param_grid = param_grid,scoring='accuracy',
                          cv = 2, n_jobs = -1, verbose = 2)
grid_search2.fit(X, Y)

print(grid_search2.best_params_)
best_grid = grid_search2.best_estimator_


Fitting 2 folds for each of 25 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


{'max_depth': 90, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [62]:
grid_search2.best_score_ #0.947

0.9478

# 엑스트라 트리

In [ ]:
#{'max_depth': 40, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

In [39]:

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300,num=3)]



# Maximum number of levels in tree
max_depth = [40,50,60,70,80,90,100]

# Minimum number of samples required to split a node
min_samples_split = [2]#2,5,10

# Minimum number of samples required at each leaf node
min_samples_leaf = [1] #2,5,10


# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

# Create a based model
clf = ExtraTreesClassifier()

# Instantiate the grid search model
grid_search3 = GridSearchCV(estimator = clf, param_grid = param_grid, scoring='accuracy'
                          cv = 2, n_jobs = -1, verbose = 2)
grid_search3.fit(X, Y)
print(grid_search3.best_params_)
best_grid = grid_search3.best_estimator_
best_score=grid_search3.best_score_

Fitting 2 folds for each of 21 candidates, totalling 42 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:   41.6s finished


{'max_depth': 40, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [41]:
best_grid

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=40, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

# light gbm 

In [ ]:
# best parameter
#{'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.005, 'n_estimators': 1400, 'num_leaves': 90, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.7}

In [86]:

from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

# Create the random grid
param_grid = { 'learning_rate': [0.005], #0.1,0.001
    'n_estimators': [1400], # 높일수록 올라가는데 어느정도 되면 큰 차이 없음
    'num_leaves' :[30,50,90] 
best_score=grid_search3.best_score_


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


{'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.005, 'n_estimators': 1400, 'num_leaves': 90, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.7}


In [87]:
best_score

0.9514

# cat boost

In [1]:
from sklearn.model_selection import GridSearchCV
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]


params = {'depth': [7,10], 
          'learning_rate' : [0.15,], #0.03, 0.1
         'l2_leaf_reg': [1,4], #4,9
         'iterations': [600]}



# Create a based model
clf = cb.CatBoostClassifier()

# Instantiate the grid search model
grid_search3 = GridSearchCV(estimator = clf, param_grid = params, scoring='accuracy',
                          cv = 2, n_jobs = -1, verbose = 2)
grid_search3.fit(X, Y)
print(grid_search3.best_params_)
best_grid = grid_search3.best_estimator_
best_score=grid_search3.best_score_

# cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
# cb_model.fit(train, y_train)

# With Categorical features
# clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
# clf.fit(train,y_train)

NameError: name 'X' is not defined

In [ ]:
best_score 